# Lab 4: Problem set on Newton-Cotes formulas

In [160]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/tma4215.css", "r").read()
    return HTML(styles)

# Comment out next line and execute this cell to restore the default notebook style 
#css_styling()

Inserting my secret latex macros here ...
$$
\DeclareMathOperator{\Div}{div}
\DeclareMathOperator{\Grad}{grad}
\DeclareMathOperator{\Curl}{curl}
\DeclareMathOperator{\Rot}{rot}
\DeclareMathOperator{\ord}{ord}
\DeclareMathOperator{\Kern}{ker}
\DeclareMathOperator{\Image}{im}
\DeclareMathOperator{\spann}{span}
\DeclareMathOperator{\rank}{rank}
\DeclareMathOperator{\dist}{dist}
\DeclareMathOperator{\diam}{diam}
\DeclareMathOperator{\sig}{sig}
\DeclareMathOperator{\Id}{Id}
\DeclareMathOperator{\CQR}{CQR}
\DeclareMathOperator{\QR}{QR}
\DeclareMathOperator{\TR}{TR}
\DeclareMathOperator{\CTR}{CTR}
\DeclareMathOperator{\SR}{SR}
\DeclareMathOperator{\CSR}{CSR}
\DeclareMathOperator{\NCR}{NCR}
\DeclareMathOperator{\MR}{MR}
\newcommand{\RR}{\mathbb{R}}
\newcommand{\NN}{\mathbb{N}}
\newcommand{\VV}{\mathbb{V}}
\newcommand{\dGamma}{\,\mathrm{d} \Gamma}
\newcommand{\dGammah}{\,\mathrm{d} \Gamma_h}
\newcommand{\dx}{\,\mathrm{d}x}
\newcommand{\dy}{\,\mathrm{d}y}
\newcommand{\ds}{\,\mathrm{d}s}
\newcommand{\dt}{\,\mathrm{d}t}
\newcommand{\dS}{\,\mathrm{d}S}
\newcommand{\dV}{\,\mathrm{d}V}
\newcommand{\dX}{\,\mathrm{d}X}
\newcommand{\dY}{\,\mathrm{d}Y}
\newcommand{\dE}{\,\mathrm{d}E}
\newcommand{\dK}{\,\mathrm{d}K}
\newcommand{\dM}{\,\mathrm{d}M}
\newcommand{\cd}{\mathrm{cd}}
\newcommand{\onehalf}{\frac{1}{2}}
\newcommand{\bfP}{\boldsymbol P}
\newcommand{\bfx}{\boldsymbol x}
\newcommand{\bfy}{\boldsymbol y}
\newcommand{\bfa}{\boldsymbol a}
\newcommand{\bfu}{\boldsymbol u}
\newcommand{\bfv}{\boldsymbol v}
\newcommand{\bfe}{\boldsymbol e}
\newcommand{\bfb}{\boldsymbol b}
\newcommand{\bfc}{\boldsymbol c}
\newcommand{\bfq}{\boldsymbol q}
\newcommand{\bfy}{\boldsymbol y}
\newcommand{\bff}{\boldsymbol f}
\newcommand{\bfp}{\boldsymbol p}
\newcommand{\bft}{\boldsymbol t}
\newcommand{\bfj}{\boldsymbol j}
\newcommand{\bfB}{\boldsymbol B}
\newcommand{\bfV}{\boldsymbol V}
\newcommand{\bfE}{\boldsymbol E}
\newcommand{\bfB}{\boldsymbol B}
\newcommand{\bfzero}{\boldsymbol 0}
$$

The main task of this problem set is the following: for a given interval $(a,b)$ and $n+1$ equally distributed nodes $x_i = a + i \tfrac{(b-a)}{n}$
for $i=0,\ldots n$, tabulate the weights for the Newton-Cotes formula up to $n=14$.
The problem boils down to 2 task, namely 

1. Defining the Lagrange polynomials $L_{in}$ for $i=0, \ldots, n$
2. Computing the weights $w_i = \int_a^b L_{in}(x) \dx$

__Before you start:__ For the implementation of the task, we recommend to use the [sympy](https://docs.sympy.org/latest/index.html#) python module  for symbolic mathematics to perform tasks such as (symbolic) integration.
Spend some time to browse  through the  [sympy tutorial](https://docs.sympy.org/latest/tutorial/index.html) and the [help on symbolic integration](https://docs.sympy.org/latest/modules/integrals/integrals.html). Using ```sympy```__Before you start:__ For the implementation of the task, we recommend to use the [sympy](https://docs.sympy.org/latest/index.html#) python module  for symbolic mathematics to perform tasks such as (symbolic) integration.
Spend some time to browse  through the  [sympy tutorial](https://docs.sympy.org/latest/tutorial/index.html) and the [help on symbolic integration](https://docs.sympy.org/latest/modules/integrals/integrals.html). Using ```sympy``` you can for instance do something like this for $n=2$:


In [161]:
# import symbol x from sympy so that you can define symbolic functions of x
from sympy.abc import x
# import symbolic integration
from sympy import integrate, cos, N
import numpy as np
from functools import reduce
import pandas as pd
import mpmath

# Define
a, b = 0, 1 
xqs = np.linspace(a,b,3)
# Define L_02 (not normalized)
L_02 = (x-xqs[1])*(x-xqs[2])
# Normalize it to satisfy l_02(x_0) = 1
L_02 = L_02/L_02.subs(x,xqs[0])

# Now integrate L_02 to compute the first weight
w_0 = integrate(L_02, (x, a, b))
print("w0 = {}".format(L_02))
print("w0 = {}".format(w_0))

w0 = 2.0*(x - 1.0)*(x - 0.5)
w0 = 0.166666666666667


Of course, 
since you are asked to do comupute all $n+1$ weights  $n=1,\ldots,14$,
you need to automatize the construction of the corresponding Lagrange polynoms.
So proceed as follows


__a__) Write  a python function ```lagrange_polys``` which takes a list of $n+1$ quadrature points
and returns a list of the corresponding $n+1$ Lagrange polynoms $\{L_{in}\}_{i=0}^n$
defined a symbolic function using ```sympy```:

In [162]:
def lagrange_polys(xqs):
    n = len(xqs)
    Ls = []
    for k in range(n):
        num = 1
        denom = 1
        for i in range(n):
            if i != k:
                num *= (x - xqs[i])
                denom *= (xqs[k] - xqs[i])
        Ls.append(num/denom)
    return Ls

# Test the function
Lg_polys = lagrange_polys(xqs)
w_0 = integrate(Lg_polys[0], (x, a, b))
print("L_02 = {}".format(Lg_polys[0]))
print("w_0 = {0:.5f}".format(w_0))

# Now for the 15 quadrature points
xqs = np.linspace(a, b, 15)
Lg_polys = lagrange_polys(xqs)

L_02 = 2.0*(x - 1.0)*(x - 0.5)
w_0 = 0.16667



__b__) Now the easy part! Employ your brand new ```def lagrange_polys``` function and implement a python function which takes as argument the desired degree of exactness $n$ and the interval end points
$a,b$ and returns a list of quadrature points $\{x_i\}_{i=0}^n$ and quadrature weights $\{w_i\}_{i=0}^n$:

In [163]:
def newton_cotes_formula(n, a, b):
    xqs = np.linspace(a, b, n+1)
    lg_polys = lagrange_polys(xqs)
    ws = [integrate(poly, (x, a, b)) for poly in lg_polys]
    return (xqs, ws)

__c__) Before you tabulate the quadrature weights with you newly implemented function, make sure that you implement them correctly. More, specifically, check for $n=1,\ldots 14$
that the computed Newton-Cotes formula integrates polynomials up to order $n$ __exactly__.

For $n$ is even, check that the corresponding Newton-Cotes rules even integrate polynomials up to order $n+1$  exactly (and not only up to $n$).

Note:  Due to floating point related errors and some numerical instabilities when computing
higher order Lagrange polynomials and integrals, the difference between the exact integral 
and the numerically error won't be 0, but around the machine precision for $n=1,2$ and then
for each increase of the order $n$ you will roughly loose of significant digit in
the difference between the two.

It might be useful to implement a little function ```qr``` first,
which takes $f$, $\{x_i\}_{i=0}^n$ and $\{w_i\}_{i=0}^n$ and
applies the corresponding quadrature to compute $\int_a^b f\dx$ numerically.


In [164]:
def qr(f, poly_order, xqs, ws):
    qr_f = 0 
    for i in range(len(ws)):
        if f.__code__.co_argcount == 1: 
            qr_f += f(xqs[i])*ws[i]
        else:
            qr_f += f(xqs[i], poly_order)*ws[i]
    return qr_f

In [165]:
mono = lambda x,m: x**m

err_n = [None]
err_np1 = [None]
w = []
for n in range(1, 15):
    args = newton_cotes_formula(n, a, b)
    err_n.append(integrate(mono(x, n), (x, a, b)) - qr(mono, n, *args))
    err_np1.append(integrate(mono(x, n + 1), (x, a, b)) - qr(mono, n + 1, *args))


err_df = pd.DataFrame({r'$E_n(x^n)$': err_n, r'$E_n(x^{n+1})$': err_np1, r'$w_i$': args[1]}) 
err_df


,$E_n(x^n)$,$E_n(x^{n+1})$,$w_i$
0,None,None,0.0180344708351186
1,0,-0.166666666666667,0.142087799933506
2,-5.55111512312578e-17,-5.55111512312578e-17,-0.154025358147919
3,1.11022302462516e-16,-0.00370370370370371,0.699748912411451
4,-1.72084568816899e-15,-1.36002320516582e-15,-1.32399751245975
5,-5.30131494258512e-15,-0.000209523809527823,2.52407717704773
6,4.04676292475870e-14,3.29042348923281e-14,-3.35786418616772
7,1.99146255042137e-13,-1.57719618204821e-5,3.90387640148401
8,2.40155118014229e-13,2.20143347995361e-13,-3.35786412656307
9,-4.25950941185249e-13,-1.37007627844243e-6,2.52407744526863


__d__) Tabulate the quadrature weights for the Newton-Cotes rule for $n=1,\ldots 14$. For which $n$  should you
refrain from using the resulting quadrature rule (and why?)

The quadrature weights, $\{w_i\}_{i=0}^n$, for the Newton-Cotes rule for $n = 1,..., 14$ are seen in the rightmost column in the output above. We see that for even $n$ the Newton-Cotes rule integrate polynomials up to order $n+1$ exactly. 

<font color='red'> For which n should you refrain from using the resulting quadrature rule (and why?)....</font>

__e__) Finally, since we have all the nice machinery in place, we take a little extra-tour
and investigate the convergence of the Newton-Cotes rules for $n\to \infty$.

More precisely, compute for $f(x) = \cos(x)$ the integral $\int_{-4}^{5} f(x) \dx $ first analytically 
and then numerically using the Newton-Cotes rules for $n=1,\ldots, 14$ and tabulate
the quadrature error $E_n(f) = \int_{-4}^{5} \cos(x)\dx - \NCR(\cos, n)$.

Finally, repeat the same experiment for for $f(x) = \tfrac{1}{1+x^2}$.

In [167]:
f_cos = lambda x: cos(x)
f_frac = lambda x: 1/(1 + x**2)
a = -4
b = 5
err_cos = []
err_frac = []

for n in range(1, 15):
    args = newton_cotes_formula(n, a, b)
    q_cos = qr(f_cos, 1, *args)
    q_frac = qr(f_frac, 1, *args)
    err_cos.append(N(integrate(f_cos(x), (x, a, b)) - q_cos, 20))
    err_frac.append(N(integrate(f_frac(x), (x, a, b)) - q_frac, 20))

errs_df = pd.DataFrame({r'$E_n\big(\cos(x)\big)$': err_cos, r'$E_n\bigg(\frac{1}{1-x^2}\bigg)$':err_frac})
errs_df.index += 1
errs_df
    

,$E_n\big(\cos(x)\big)$,$E_n\bigg(\frac{1}{1-x^2}\bigg)$
1,-0.050810310669331124067,2.2614356251831840735
2,-6.4262499882127245926,-2.2467091711969066173
3,-1.7185223641039987869,0.22727272925558260699
4,1.0183161502430147421,0.50970394767220460555
5,0.51827111772295202662,0.15675530325242658183
6,-0.12533837970122236606,-0.37132097678194287890
7,-0.073176252487429713233,-0.21459589706481997293
8,0.010600086697775171197,0.29883127337323755004
9,0.0065893125627208198773,0.15694095808579776677
10,-0.00064955371771242277309,0.012324200963466636875
